# 使用PaddleNLP生成式API搭建一个聊天机器人

近年来，人机对话系统受到了学术界和产业界的广泛关注并取得了不错的发展。开放域对话系统旨在建立一个开放域的多轮对话系统，使得机器可以流畅自然地与人进行语言交互，既可以进行日常问候类的闲聊，又可以完成特定功能，以使得开放域对话系统具有实际应用价值。

本实例将重点介绍PaddleNLP内置的**生成式API**的功能和用法，并使用PaddleNLP内置的**plato-mini模型**和配套的**生成式API**实现一个简单的闲聊机器人。

## 环境要求

* PaddlePaddle

   本项目依赖于 PaddlePaddle 2.0 及以上版本，请参考 [安装指南](https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/pip/windows-pip.html) 进行安装

* PaddleNLP 

   ```shell
   pip install --upgrade paddlenlp -i https://pypi.org/simple
   ```
* sentencepiece 

   ```shell
   pip install --upgrade sentencepiece -i https://pypi.org/simple
   ```

* Python

    Python的版本要求 3.6+

AI Studio平台后续会默认安装PaddleNLP，在此之前可使用如下命令安装

In [1]:
!pip install --upgrade paddlenlp -i https://pypi.org/simple

     |████████████████████████████████| 440kB 41kB/s eta 0:00:012
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1


In [2]:
!pip install --upgrade pip
!pip install --upgrade sentencepiece 

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.6MB 15.0MB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.2 MB 20.8 MB/s eta 0:00:01
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.85
    Uninstalling sentencepiece-0.1.85:
      Successfully uninstalled sentencepiece-0.1.85


## 生成式API

**PaddleNLP针对生成式任务提供了`generate()`函数，内嵌于PaddleNLP所有的生成式模型。支持Greedy Search、Beam Search和Sampling解码策略，用户只需指定解码策略以及相应的参数即可完成预测解码，得到生成的sequence的token ids以及概率得分。**

下面给大家展示一个GPT模型使用生成API的例子：

## 1. 加载 `paddlenlp.transformers.GPTChineseTokenizer`用于数据处理

文本数据在输入预训练模型之前，需要经过数据处理转化为Feature。这一过程通常包括分词，token to id，add special token等步骤。  

**PaddleNLP对于各种预训练模型已经内置了相应的tokenizer**，指定想要使用的模型名字即可加载对应的tokenizer。

调用`GPTChineseTokenizer`的`__call__`方法即可将我们说的话转为模型可接受的输入。

In [3]:
from paddlenlp.transformers import GPTChineseTokenizer

# 设置想要使用模型的名称
model_name = 'gpt-cpm-small-cn-distill'
tokenizer = GPTChineseTokenizer.from_pretrained(model_name)

[2021-06-25 23:31:34,283] [    INFO] - Downloading gpt-cpm-cn-sentencepiece.model from https://paddlenlp.bj.bcebos.com/models/transformers/gpt/gpt-cpm-cn-sentencepiece.model
100%|██████████| 697/697 [00:00<00:00, 8828.89it/s]


In [8]:
import paddle

user_input = "莫愁前路无知己，"

# 将文本转为ids
input_ids = tokenizer(user_input)['input_ids']
print(input_ids)

# 将转换好的id转为tensor
input_ids = paddle.to_tensor(input_ids, dtype='int64').unsqueeze(0)
print(input_ids)

[2683, 6102, 241, 400, 7962, 3357, 8, 9]
Tensor(shape=[1, 8], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [[2683, 6102, 241 , 400 , 7962, 3357, 8   , 9   ]])


## 2. 使用PaddleNLP一键加载预训练模型

**PaddleNLP提供了GPT,UnifiedTransformer等中文预训练模型，可以通过预训练模型名称完成一键加载。**

GPT以Transformer Decoder的编码器为网络基本组件，采用单向注意力机制，适用于长文本生成任务。

PaddleNLP目前提供多种中英文GPT预训练模型，我们这次用的是一个小的中文GPT预训练模型。其他预训练模型请参考[模型列表](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/transformers.html)。

In [9]:
from paddlenlp.transformers import GPTLMHeadModel

# 一键加载中文GPT模型
model = GPTLMHeadModel.from_pretrained(model_name)

[2021-06-25 23:33:26,544] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/gpt-cpm-small-cn-distill/gpt-cpm-small-cn-distill.pdparams


ValueError: parameter name [word_embeddings] have be been used. In dygraph mode, the name of parameter can't be same.Please check the parameter attr value passed to self.create_parameter or constructor of dygraph Layers

In [10]:
# 调用生成API升成文本
ids, scores = model.generate(
                input_ids=input_ids,
                max_length=16,
                min_length=1,
                decode_strategy='greedy_search')

print(ids)
print(scores)

Tensor(shape=[1, 16], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [[8 , 10, 8 , 10, 8 , 10, 8 , 10, 8 , 10, 8 , 10, 8 , 10, 8 , 10]])
Tensor(shape=[1, 1], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[-0.57972622]])


In [12]:
generated_ids = ids[0].numpy().tolist()

# 使用tokenizer将生成的id转为文本
generated_text = tokenizer.convert_ids_to_string(generated_ids)
print(generated_text)

可以看到生成的效果还不错，生成式API的用法也是非常的简便。

下面我们来展示一下如何使用UnifiedTransformer模型和生成式API完成闲聊对话。

## 1. 加载 `paddlenlp.transformers.UnifiedTransformerTokenizer`用于数据处理

`UnifiedTransformerTokenizer`的调用方式与GPT相同，但数据处理的API略有不同。

调用`UnifiedTransformerTokenizer`的`dialogue_encode`方法即可将我们说的话转为模型可接受的输入。

In [13]:
from paddlenlp.transformers import UnifiedTransformerTokenizer

# 设置想要使用模型的名称
model_name = 'plato-mini'
tokenizer = UnifiedTransformerTokenizer.from_pretrained(model_name)

[2021-06-25 23:33:47,981] [    INFO] - Downloading plato-mini-vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini-vocab.txt
100%|██████████| 410/410 [00:00<00:00, 3748.39it/s]
[2021-06-25 23:33:48,182] [    INFO] - Downloading plato-mini-spm.model from https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini-spm.model
100%|██████████| 712/712 [00:00<00:00, 9046.65it/s]


In [14]:
user_input = ['你好啊，你今年多大了']

# 调用dialogue_encode方法生成输入
encoded_input = tokenizer.dialogue_encode(
                    user_input,
                    add_start_token_as_response=True,
                    return_tensors=True,
                    is_split_into_words=False)

print(encoded_input.keys())

dict_keys(['input_ids', 'token_type_ids', 'position_ids', 'attention_mask'])


`dialogue_encode`的详细用法，请参考[dialogue_encode](https://paddlenlp.readthedocs.io/zh/latest/source/paddlenlp.transformers.unified_transformer.tokenizer.html#paddlenlp.transformers.unified_transformer.tokenizer.UnifiedTransformerTokenizer.dialogue_encode)。

## 2. 使用PaddleNLP一键加载预训练模型

与GPT相同，我们可以一键调用UnifiedTransformer预训练模型。

[UnifiedTransformer](https://github.com/PaddlePaddle/Knover/tree/luge-dialogue/luge-dialogue)以Transformer的编码器为网络基本组件，采用灵活的注意力机制，十分适合文本生成任务，并在模型输入中加入了标识不同对话技能的special token，使得模型能同时支持闲聊对话、推荐对话和知识对话。

PaddleNLP目前为UnifiedTransformer提供了三个中文预训练模型：
- `unified_transformer-12L-cn` 该预训练模型是在大规模中文会话数据集上训练得到的
- `unified_transformer-12L-cn-luge` 该预训练模型是`unified_transformer-12L-cn`在千言对话数据集上进行微调得到的。
- `plato-mini` 该模型使用了十亿级别的中文闲聊对话数据进行预训练。

In [15]:
from paddlenlp.transformers import UnifiedTransformerLMHeadModel

model = UnifiedTransformerLMHeadModel.from_pretrained(model_name)

[2021-06-25 23:34:33,400] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini.pdparams and saved to /home/aistudio/.paddlenlp/models/plato-mini
[2021-06-25 23:34:33,403] [    INFO] - Downloading plato-mini.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini.pdparams
100%|██████████| 530157/530157 [00:09<00:00, 57237.13it/s]


下一步我们将处理好的输入传入generate函数，并配置解码策略。

这里我们使用的是TopK加sampling的解码策略。即从概率最大的k个结果中按概率进行采样。

In [16]:
ids, scores = model.generate(
                input_ids=encoded_input['input_ids'],
                token_type_ids=encoded_input['token_type_ids'],
                position_ids=encoded_input['position_ids'],
                attention_mask=encoded_input['attention_mask'],
                max_length=64,
                min_length=1,
                decode_strategy='sampling',
                top_k=5,
                num_return_sequences=20)

print(ids)
print(scores)

Tensor(shape=[20, 15], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [[912 , 28  , 3   , 6   , 763 , 1585, 26028, 197 , 3   , 9   , 94  , 16  , 2   , 0   , 0   ],
        [6   , 763 , 908 , 26028, 7   , 3   , 6   , 804 , 37  , 2476, 3   , 9   , 94  , 16  , 2   ],
        [6   , 763 , 1585, 26028, 2   , 0   , 0   , 0   , 0   , 0   , 0   , 0   , 0   , 0   , 0   ],
        [6   , 763 , 1164, 26028, 7   , 3   , 9   , 42  , 25375, 28  , 16  , 2   , 0   , 0   , 0   ],
        [6   , 763 , 1164, 26028, 3   , 9   , 42  , 25375, 7   , 94  , 16  , 2   , 0   , 0   , 0   ],
        [6   , 28  , 763 , 1850, 26028, 197 , 2   , 0   , 0   , 0   , 0   , 0   , 0   , 0   , 0   ],
        [6   , 763 , 1164, 7   , 3   , 215 , 481 , 17  , 180 , 25397, 25549, 7   , 2   , 0   , 0   ],
        [6   , 763 , 1585, 26028, 7   , 2   , 0   , 0   , 0   , 0   , 0   , 0   , 0   , 0   , 0   ],
        [6   , 763 , 1585, 26028, 197 , 2   , 0   , 0   , 0   , 0   , 0   , 0   , 0   , 0   , 0   ],
        [6  

In [17]:
from utils import select_response

# 简单根据概率选取最佳回复
result = select_response(ids, scores, tokenizer, keep_space=False, num_return_sequences=20)
print(result)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlenlp/data/vocab.py:220: UserWarning: The type of `to_tokens()`'s input `indices` is not `int` which will be forcibly transfered to `int`. 
  "The type of `to_tokens()`'s input `indices` is not `int` which will be forcibly transfered to `int`. "


['我今年23岁了,你多大啊?']


关于生成式API更详细的用法，请参考[generate](https://paddlenlp.readthedocs.io/zh/latest/source/paddlenlp.transformers.generation_utils.html#paddlenlp.transformers.generation_utils.GenerationMixin.generate)。

## 下面我们去终端里试试多轮对话吧！

PaddleNLP的example中提供了搭建完整对话系统的代码（[人机交互](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples/dialogue/unified_transformer#%E4%BA%BA%E6%9C%BA%E4%BA%A4%E4%BA%92)）。我们可以去终端里尝试一下。

# Paddle+Wechaty

开头我们展示的聊天机器人就是使用PaddleHub和Wechaty搭建的。大家可以参照这个教程自己尝试[paddlehub-wechaty-demo](https://github.com/KPatr1ck/paddlehub-wechaty-demo)。

以上内容实现基于PaddleNLP，开源不易，希望大家多多支持~ 
**记得给[PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)点个小小的Star⭐**

GitHub地址：[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)
![](https://ai-studio-static-online.cdn.bcebos.com/a0e8ca7743ea4fe9aa741682a63e767f8c48dc55981f4e44a40e0e00d3ab369e)

**更多使用方法可参考PaddleNLP教程**

- [使用seq2vec模块进行句子情感分类](https://aistudio.baidu.com/aistudio/projectdetail/1283423)
- [使用预训练模型ERNIE优化情感分析](https://aistudio.baidu.com/aistudio/projectdetail/1294333)
- [使用BiGRU-CRF模型完成快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1317771)
- [使用预训练模型ERNIE优化快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1329361)
- [使用Seq2Seq模型完成自动对联](https://aistudio.baidu.com/aistudio/projectdetail/1321118)
- [使用预训练模型ERNIE-GEN自动写诗](https://aistudio.baidu.com/aistudio/projectdetail/1339888)
- [使用TCN网络完成新冠疫情病例数预测](https://aistudio.baidu.com/aistudio/projectdetail/1290873)
- [自定义数据集实现文本多分类任务](https://aistudio.baidu.com/aistudio/projectdetail/1468469)

# 加入交流群，一起学习吧

现在就加入PaddleNLP的QQ技术交流群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >